# End to End Pure Streaming Data-Pipeline for Tenant Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read tenant state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "tenant_streamingDF" --> "results_df" --> "tenant_data"

In [0]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [0]:
# Schema for Tenant
tenant_schema = StructType([
            StructField("Tenant_id", IntegerType(), False),
            StructField("First_name", StringType(), True),
            StructField("Last_name", StringType(), False),
            StructField("Ssn", StringType(), True),
            StructField("Phone", StringType(), True),
            StructField("Email", StringType(), True), 
            StructField("Mobile", StringType(), True),
            StructField("event_time", TimestampType(), True)])

tenant_udf_schema = StructType([
            StructField("First_name", StringType(), True),
            StructField("Last_name", StringType(), False),
            StructField("Ssn", StringType(), True),
            StructField("Phone", StringType(), True),
            StructField("Email", StringType(), True), 
            StructField("Mobile", StringType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get tenant csv files as a streaming "tenant_streamingDF" and process it on the fly and get transformed stream "tenant_df"
###### Objective: (incoming csv files) --> "tenant_streamingDF" --> "tenant_df"

In [0]:
# Get tenant Steaming DataFrame from csv files

# streaming starts here by reading the input files 
tenant_Path = "/FileStore/apartment/tenant/inprogress/"
tenant_streamingDF = (
  spark
    .readStream
    .schema(tenant_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(tenant_Path)
)
# Clear invalid rows
tenant_df = tenant_streamingDF.select("*").where("Tenant_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(tenant_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
transformer.remove_special_chars(columns=['First_name', 'Last_name', 'First_name', 'Ssn'])

Out[ 5 ]: <optimus.df_transformer.DataFrameTransformer at 0x7fb436790ef0>

##### This function parses the corresponding columns into a single column

In [0]:
def my_fun(First_name, Last_name, Ssn, Phone, Email, Mobile, event_time):
  return zip(First_name, Last_name, Ssn, Phone, Email, Mobile, event_time)

udf_Fun = udf(my_fun, ArrayType(tenant_udf_schema))

In [0]:
intermediate_df = ( tenant_df.withWatermark("event_time", "10 seconds")
            .groupBy("Tenant_id")
            .agg(F.collect_list("First_name").alias("First_name"),
                 F.collect_list("Last_name").alias("Last_name"),
                 F.collect_list("Ssn").alias("Ssn"),
                 F.collect_list("Phone").alias("Phone"),
                 F.collect_list("Email").alias("Email"),
                 F.collect_list("Mobile").alias("Mobile"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Tenant_id", 
                    F.explode(udf_Fun(F.column("First_name"), 
                                      F.column("Last_name"), 
                                      F.column("Ssn"), 
                                      F.column("Phone"), 
                                      F.column("Email"),
                                      F.column("Mobile"), 
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [0]:
results_df = (intermediate_df
              .select("Tenant_id", 
                      "data.First_name", 
                      "data.Last_name", 
                      "data.Ssn", 
                      "data.Phone", 
                      "data.Email", 
                      "data.Mobile",
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Tenant_id")

##### Display final result
###### This result shows the latest state of all the unique tenant_id

In [0]:
display(results_df)

Tenant_id,First_name,Last_name,Ssn,Phone,Email,Mobile,event_time,latest_event_time
1,Humbert,Sizzey,299-75-4458,921 679 8198,hsizzey0@google.de,+244 (475) 695-5117,2017-11-17T21:50:55.000+0000,2017-11-17T21:50:55.000+0000
2,Fidelity,Cundey,202-24-0149,552 799 3302,fcundey1@europa.eu,+269 (270) 578-0662,2018-05-30T03:54:19.000+0000,2018-05-30T03:54:19.000+0000
3,Noni,Oehm,409-26-9066,404 273 6111,noehm2@parallels.com,+351 (156) 200-9652,2018-11-26T01:55:24.000+0000,2018-11-26T01:55:24.000+0000
4,Agosto,Atyea,251-68-0644,286 617 6820,aatyea3@weibo.com,+966 (468) 405-4985,2018-11-12T02:06:34.000+0000,2018-11-12T02:06:34.000+0000
5,Pyotr,Denys,200-77-1350,988 856 1497,pdenys4@rakuten.co.jp,+46 (431) 284-1398,2016-12-31T06:07:33.000+0000,2016-12-31T06:07:33.000+0000
6,Sander,Loiterton,408-11-6878,847 273 8016,sloiterton5@purevolume.com,+86 (127) 503-2589,2017-11-21T10:16:57.000+0000,2017-11-21T10:16:57.000+0000
7,Luelle,Gilliard,328-38-3523,671 917 3566,lgilliard6@fema.gov,+86 (903) 204-3977,2018-10-11T13:23:29.000+0000,2018-10-11T13:23:29.000+0000
8,Nollie,Lortz,821-09-5387,603 363 8883,nlortz7@gravatar.com,+352 (856) 280-5640,2018-06-17T07:33:42.000+0000,2018-06-17T07:33:42.000+0000
9,Salem,Buckby,544-87-4924,733 696 5652,sbuckby8@squidoo.com,+92 (116) 435-1823,2016-12-30T05:39:21.000+0000,2016-12-30T05:39:21.000+0000
10,Ernaline,Leyninye,265-70-3003,521 630 9168,eleyninye9@cdc.gov,+86 (167) 135-0401,2017-08-27T18:59:45.000+0000,2017-08-27T18:59:45.000+0000


##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [0]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/tenant/_checkpoints/streaming-agg")
 .start("/delta/apartment/tenant_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe will always be Up-To-Date

In [0]:
tenant_data = spark.read.format("delta").load("/delta/apartment/tenant_data").orderBy("Tenant_id")

In [0]:
display(tenant_data)

Tenant_id,First_name,Last_name,Ssn,Phone,Email,Mobile,event_time,latest_event_time
1,Humbert,Sizzey,299-75-4458,921 679 8198,hsizzey0@google.de,+244 (475) 695-5117,2017-11-17T21:50:55.000+0000,2017-11-17T21:50:55.000+0000
2,Fidelity,Cundey,202-24-0149,552 799 3302,fcundey1@europa.eu,+269 (270) 578-0662,2018-05-30T03:54:19.000+0000,2018-05-30T03:54:19.000+0000
3,Noni,Oehm,409-26-9066,404 273 6111,noehm2@parallels.com,+351 (156) 200-9652,2018-11-26T01:55:24.000+0000,2018-11-26T01:55:24.000+0000
4,Agosto,Atyea,251-68-0644,286 617 6820,aatyea3@weibo.com,+966 (468) 405-4985,2018-11-12T02:06:34.000+0000,2018-11-12T02:06:34.000+0000
5,Pyotr,Denys,200-77-1350,988 856 1497,pdenys4@rakuten.co.jp,+46 (431) 284-1398,2016-12-31T06:07:33.000+0000,2016-12-31T06:07:33.000+0000
6,Sander,Loiterton,408-11-6878,847 273 8016,sloiterton5@purevolume.com,+86 (127) 503-2589,2017-11-21T10:16:57.000+0000,2017-11-21T10:16:57.000+0000
7,Luelle,Gilliard,328-38-3523,671 917 3566,lgilliard6@fema.gov,+86 (903) 204-3977,2018-10-11T13:23:29.000+0000,2018-10-11T13:23:29.000+0000
8,Nollie,Lortz,821-09-5387,603 363 8883,nlortz7@gravatar.com,+352 (856) 280-5640,2018-06-17T07:33:42.000+0000,2018-06-17T07:33:42.000+0000
9,Salem,Buckby,544-87-4924,733 696 5652,sbuckby8@squidoo.com,+92 (116) 435-1823,2016-12-30T05:39:21.000+0000,2016-12-30T05:39:21.000+0000
10,Ernaline,Leyninye,265-70-3003,521 630 9168,eleyninye9@cdc.gov,+86 (167) 135-0401,2017-08-27T18:59:45.000+0000,2017-08-27T18:59:45.000+0000
